In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_excel('./combined.xlsx')
df.head()


,info,id
0,286-69-1076,1
1,256-80-1815,1
2,398-21-8862,1
3,725-50-8395,1
4,882-45-5093,1


In [3]:
df = df.sample(frac = 1) #shuffle the data

In [4]:
df

,info,id
459109,PI248441J,5
488139,HO224558T,5
357997,3002147670060700,4
134697,582358834054,2
126906,232873241099,2
...,...,...
414445,SI374382J,5
476182,KU504109B,5
443137,MI413398L,5
551509,NSC-16-352-61522,6


In [5]:
df["info"] = df["info"].astype(str)

# Verify that the values in the column are now strings
print(df["info"].dtype)

object


In [6]:
y = pd.get_dummies(df['id'], prefix='label')
y

,label_1,label_2,label_3,label_4,label_5,label_6,label_7
459109,0,0,0,0,1,0,0
488139,0,0,0,0,1,0,0
357997,0,0,0,1,0,0,0
134697,0,1,0,0,0,0,0
126906,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
414445,0,0,0,0,1,0,0
476182,0,0,0,0,1,0,0
443137,0,0,0,0,1,0,0
551509,0,0,0,0,0,1,0


In [7]:
label = list(y.columns)
label

['label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7']

In [8]:
y = y.to_dict('index')
y

{459109: {'label_1': 0,
  'label_2': 0,
  'label_3': 0,
  'label_4': 0,
  'label_5': 1,
  'label_6': 0,
  'label_7': 0},
 488139: {'label_1': 0,
  'label_2': 0,
  'label_3': 0,
  'label_4': 0,
  'label_5': 1,
  'label_6': 0,
  'label_7': 0},
 357997: {'label_1': 0,
  'label_2': 0,
  'label_3': 0,
  'label_4': 1,
  'label_5': 0,
  'label_6': 0,
  'label_7': 0},
 134697: {'label_1': 0,
  'label_2': 1,
  'label_3': 0,
  'label_4': 0,
  'label_5': 0,
  'label_6': 0,
  'label_7': 0},
 126906: {'label_1': 0,
  'label_2': 1,
  'label_3': 0,
  'label_4': 0,
  'label_5': 0,
  'label_6': 0,
  'label_7': 0},
 425473: {'label_1': 0,
  'label_2': 0,
  'label_3': 0,
  'label_4': 0,
  'label_5': 1,
  'label_6': 0,
  'label_7': 0},
 522881: {'label_1': 0,
  'label_2': 0,
  'label_3': 0,
  'label_4': 0,
  'label_5': 0,
  'label_6': 1,
  'label_7': 0},
 89311: {'label_1': 1,
  'label_2': 0,
  'label_3': 0,
  'label_4': 0,
  'label_5': 0,
  'label_6': 0,
  'label_7': 0},
 13682: {'label_1': 1,
  'label_2

In [9]:
dataset = list(zip(df['info'],[{'cats': cats} for cats in y.values()]))
print(dataset[1])

('HO224558T', {'cats': {'label_1': 0, 'label_2': 0, 'label_3': 0, 'label_4': 0, 'label_5': 1, 'label_6': 0, 'label_7': 0}})


In [10]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, train_size=0.8, random_state=13)

In [11]:
import spacy

nlp = spacy.blank("en")

textcat = nlp.add_pipe("textcat_multilabel")
for i in label:
    textcat.add_label(i)

In [12]:
textcat.labels

('label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7')

In [13]:
nlp.analyze_pipes()

{'summary': {'textcat_multilabel': {'assigns': ['doc.cats'],
   'requires': [],
   'scores': ['cats_score',
    'cats_score_desc',
    'cats_micro_p',
    'cats_micro_r',
    'cats_micro_f',
    'cats_macro_p',
    'cats_macro_r',
    'cats_macro_f',
    'cats_macro_auc',
    'cats_f_per_type'],
   'retokenizes': False}},
 'problems': {'textcat_multilabel': []},
 'attrs': {'doc.cats': {'assigns': ['textcat_multilabel'], 'requires': []}}}

In [14]:
optimizer = nlp.begin_training()
iterations = 2

In [15]:
from spacy.util import minibatch, compounding
from spacy.training import Example
with nlp.select_pipes(enable="textcat_multilabel"):
    for j in range(iterations):
        losses = {}
        k = 0
        batches = minibatch(train_data, size = compounding(4.,32.,1.001))
        for batch in batches:
            text, annotations = zip(*batch)
            example = []
            for i in range(len(text)):
                doc = nlp.make_doc(text[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, sgd=optimizer, drop=0.2, losses = losses)
            print('Batch No: {} Loss = {}'.format(k, round(losses['textcat_multilabel'])))
            k += 1
        print("\n\n Completed Iterations : {} ".format(j))

Batch No: 0 Loss = 0
Batch No: 1 Loss = 1
Batch No: 2 Loss = 1
Batch No: 3 Loss = 1
Batch No: 4 Loss = 1
Batch No: 5 Loss = 2
Batch No: 6 Loss = 2
Batch No: 7 Loss = 2
Batch No: 8 Loss = 3
Batch No: 9 Loss = 3
Batch No: 10 Loss = 3
Batch No: 11 Loss = 3
Batch No: 12 Loss = 4
Batch No: 13 Loss = 4
Batch No: 14 Loss = 4
Batch No: 15 Loss = 4
Batch No: 16 Loss = 4
Batch No: 17 Loss = 5
Batch No: 18 Loss = 5
Batch No: 19 Loss = 5
Batch No: 20 Loss = 5
Batch No: 21 Loss = 5
Batch No: 22 Loss = 6
Batch No: 23 Loss = 6
Batch No: 24 Loss = 6
Batch No: 25 Loss = 6
Batch No: 26 Loss = 6
Batch No: 27 Loss = 6
Batch No: 28 Loss = 7
Batch No: 29 Loss = 7
Batch No: 30 Loss = 7
Batch No: 31 Loss = 7
Batch No: 32 Loss = 7
Batch No: 33 Loss = 7
Batch No: 34 Loss = 8
Batch No: 35 Loss = 8
Batch No: 36 Loss = 8
Batch No: 37 Loss = 8
Batch No: 38 Loss = 8
Batch No: 39 Loss = 8
Batch No: 40 Loss = 8
Batch No: 41 Loss = 9
Batch No: 42 Loss = 9
Batch No: 43 Loss = 9
Batch No: 44 Loss = 9
Batch No: 45 Loss = 

Batch No: 360 Loss = 25
Batch No: 361 Loss = 25
Batch No: 362 Loss = 25
Batch No: 363 Loss = 25
Batch No: 364 Loss = 25
Batch No: 365 Loss = 25
Batch No: 366 Loss = 25
Batch No: 367 Loss = 25
Batch No: 368 Loss = 25
Batch No: 369 Loss = 25
Batch No: 370 Loss = 25
Batch No: 371 Loss = 25
Batch No: 372 Loss = 25
Batch No: 373 Loss = 25
Batch No: 374 Loss = 25
Batch No: 375 Loss = 26
Batch No: 376 Loss = 26
Batch No: 377 Loss = 26
Batch No: 378 Loss = 26
Batch No: 379 Loss = 26
Batch No: 380 Loss = 26
Batch No: 381 Loss = 26
Batch No: 382 Loss = 26
Batch No: 383 Loss = 26
Batch No: 384 Loss = 26
Batch No: 385 Loss = 26
Batch No: 386 Loss = 26
Batch No: 387 Loss = 26
Batch No: 388 Loss = 26
Batch No: 389 Loss = 26
Batch No: 390 Loss = 26
Batch No: 391 Loss = 26
Batch No: 392 Loss = 26
Batch No: 393 Loss = 26
Batch No: 394 Loss = 26
Batch No: 395 Loss = 26
Batch No: 396 Loss = 26
Batch No: 397 Loss = 26
Batch No: 398 Loss = 26
Batch No: 399 Loss = 26
Batch No: 400 Loss = 26
Batch No: 401 Lo

Batch No: 707 Loss = 31
Batch No: 708 Loss = 31
Batch No: 709 Loss = 31
Batch No: 710 Loss = 31
Batch No: 711 Loss = 31
Batch No: 712 Loss = 31
Batch No: 713 Loss = 31
Batch No: 714 Loss = 31
Batch No: 715 Loss = 31
Batch No: 716 Loss = 31
Batch No: 717 Loss = 32
Batch No: 718 Loss = 32
Batch No: 719 Loss = 32
Batch No: 720 Loss = 32
Batch No: 721 Loss = 32
Batch No: 722 Loss = 32
Batch No: 723 Loss = 32
Batch No: 724 Loss = 32
Batch No: 725 Loss = 32
Batch No: 726 Loss = 32
Batch No: 727 Loss = 32
Batch No: 728 Loss = 32
Batch No: 729 Loss = 32
Batch No: 730 Loss = 32
Batch No: 731 Loss = 32
Batch No: 732 Loss = 32
Batch No: 733 Loss = 32
Batch No: 734 Loss = 32
Batch No: 735 Loss = 32
Batch No: 736 Loss = 32
Batch No: 737 Loss = 32
Batch No: 738 Loss = 32
Batch No: 739 Loss = 32
Batch No: 740 Loss = 32
Batch No: 741 Loss = 32
Batch No: 742 Loss = 32
Batch No: 743 Loss = 32
Batch No: 744 Loss = 32
Batch No: 745 Loss = 32
Batch No: 746 Loss = 32
Batch No: 747 Loss = 32
Batch No: 748 Lo

Batch No: 1054 Loss = 36
Batch No: 1055 Loss = 36
Batch No: 1056 Loss = 36
Batch No: 1057 Loss = 36
Batch No: 1058 Loss = 36
Batch No: 1059 Loss = 36
Batch No: 1060 Loss = 36
Batch No: 1061 Loss = 36
Batch No: 1062 Loss = 36
Batch No: 1063 Loss = 36
Batch No: 1064 Loss = 36
Batch No: 1065 Loss = 36
Batch No: 1066 Loss = 36
Batch No: 1067 Loss = 36
Batch No: 1068 Loss = 36
Batch No: 1069 Loss = 36
Batch No: 1070 Loss = 36
Batch No: 1071 Loss = 36
Batch No: 1072 Loss = 36
Batch No: 1073 Loss = 36
Batch No: 1074 Loss = 36
Batch No: 1075 Loss = 36
Batch No: 1076 Loss = 36
Batch No: 1077 Loss = 36
Batch No: 1078 Loss = 36
Batch No: 1079 Loss = 36
Batch No: 1080 Loss = 36
Batch No: 1081 Loss = 36
Batch No: 1082 Loss = 36
Batch No: 1083 Loss = 36
Batch No: 1084 Loss = 36
Batch No: 1085 Loss = 36
Batch No: 1086 Loss = 36
Batch No: 1087 Loss = 36
Batch No: 1088 Loss = 36
Batch No: 1089 Loss = 36
Batch No: 1090 Loss = 36
Batch No: 1091 Loss = 36
Batch No: 1092 Loss = 36
Batch No: 1093 Loss = 36


Batch No: 1383 Loss = 38
Batch No: 1384 Loss = 38
Batch No: 1385 Loss = 38
Batch No: 1386 Loss = 38
Batch No: 1387 Loss = 38
Batch No: 1388 Loss = 38
Batch No: 1389 Loss = 38
Batch No: 1390 Loss = 38
Batch No: 1391 Loss = 38
Batch No: 1392 Loss = 38
Batch No: 1393 Loss = 38
Batch No: 1394 Loss = 38
Batch No: 1395 Loss = 38
Batch No: 1396 Loss = 38
Batch No: 1397 Loss = 38
Batch No: 1398 Loss = 38
Batch No: 1399 Loss = 38
Batch No: 1400 Loss = 38
Batch No: 1401 Loss = 38
Batch No: 1402 Loss = 38
Batch No: 1403 Loss = 38
Batch No: 1404 Loss = 38
Batch No: 1405 Loss = 38
Batch No: 1406 Loss = 38
Batch No: 1407 Loss = 38
Batch No: 1408 Loss = 38
Batch No: 1409 Loss = 38
Batch No: 1410 Loss = 38
Batch No: 1411 Loss = 38
Batch No: 1412 Loss = 38
Batch No: 1413 Loss = 38
Batch No: 1414 Loss = 38
Batch No: 1415 Loss = 38
Batch No: 1416 Loss = 38
Batch No: 1417 Loss = 38
Batch No: 1418 Loss = 38
Batch No: 1419 Loss = 38
Batch No: 1420 Loss = 38
Batch No: 1421 Loss = 38
Batch No: 1422 Loss = 38


...

Batch No: 17145 Loss = 40
Batch No: 17146 Loss = 40
Batch No: 17147 Loss = 40
Batch No: 17148 Loss = 40
Batch No: 17149 Loss = 40
Batch No: 17150 Loss = 40
Batch No: 17151 Loss = 40
Batch No: 17152 Loss = 40
Batch No: 17153 Loss = 40
Batch No: 17154 Loss = 40
Batch No: 17155 Loss = 40
Batch No: 17156 Loss = 40
Batch No: 17157 Loss = 40
Batch No: 17158 Loss = 40
Batch No: 17159 Loss = 40
Batch No: 17160 Loss = 40
Batch No: 17161 Loss = 40
Batch No: 17162 Loss = 40
Batch No: 17163 Loss = 40
Batch No: 17164 Loss = 40
Batch No: 17165 Loss = 40
Batch No: 17166 Loss = 40
Batch No: 17167 Loss = 40
Batch No: 17168 Loss = 40
Batch No: 17169 Loss = 40
Batch No: 17170 Loss = 40
Batch No: 17171 Loss = 40
Batch No: 17172 Loss = 40
Batch No: 17173 Loss = 40
Batch No: 17174 Loss = 40
Batch No: 17175 Loss = 40
Batch No: 17176 Loss = 40
Batch No: 17177 Loss = 40
Batch No: 17178 Loss = 40
Batch No: 17179 Loss = 40
Batch No: 17180 Loss = 40
Batch No: 17181 Loss = 40
Batch No: 17182 Loss = 40
Batch No: 17

Batch No: 17462 Loss = 41
Batch No: 17463 Loss = 41
Batch No: 17464 Loss = 41
Batch No: 17465 Loss = 41
Batch No: 17466 Loss = 41
Batch No: 17467 Loss = 41
Batch No: 17468 Loss = 41
Batch No: 17469 Loss = 41
Batch No: 17470 Loss = 41
Batch No: 17471 Loss = 41
Batch No: 17472 Loss = 41
Batch No: 17473 Loss = 41
Batch No: 17474 Loss = 41
Batch No: 17475 Loss = 41
Batch No: 17476 Loss = 41
Batch No: 17477 Loss = 41
Batch No: 17478 Loss = 41
Batch No: 17479 Loss = 41
Batch No: 17480 Loss = 41
Batch No: 17481 Loss = 41
Batch No: 17482 Loss = 41
Batch No: 17483 Loss = 41
Batch No: 17484 Loss = 41
Batch No: 17485 Loss = 41
Batch No: 17486 Loss = 41
Batch No: 17487 Loss = 41
Batch No: 17488 Loss = 41
Batch No: 17489 Loss = 41
Batch No: 17490 Loss = 41
Batch No: 17491 Loss = 41
Batch No: 17492 Loss = 41
Batch No: 17493 Loss = 41
Batch No: 17494 Loss = 41
Batch No: 17495 Loss = 41
Batch No: 17496 Loss = 41
Batch No: 17497 Loss = 41
Batch No: 17498 Loss = 41
Batch No: 17499 Loss = 41
Batch No: 17

Batch No: 17781 Loss = 42
Batch No: 17782 Loss = 42
Batch No: 17783 Loss = 42
Batch No: 17784 Loss = 42
Batch No: 17785 Loss = 42
Batch No: 17786 Loss = 42
Batch No: 17787 Loss = 42
Batch No: 17788 Loss = 42
Batch No: 17789 Loss = 42
Batch No: 17790 Loss = 42
Batch No: 17791 Loss = 42
Batch No: 17792 Loss = 42
Batch No: 17793 Loss = 42
Batch No: 17794 Loss = 42
Batch No: 17795 Loss = 42
Batch No: 17796 Loss = 42
Batch No: 17797 Loss = 42
Batch No: 17798 Loss = 42
Batch No: 17799 Loss = 42
Batch No: 17800 Loss = 42
Batch No: 17801 Loss = 42
Batch No: 17802 Loss = 42
Batch No: 17803 Loss = 42
Batch No: 17804 Loss = 42
Batch No: 17805 Loss = 42
Batch No: 17806 Loss = 42
Batch No: 17807 Loss = 42
Batch No: 17808 Loss = 42
Batch No: 17809 Loss = 42
Batch No: 17810 Loss = 42
Batch No: 17811 Loss = 42
Batch No: 17812 Loss = 42
Batch No: 17813 Loss = 42
Batch No: 17814 Loss = 42
Batch No: 17815 Loss = 42
Batch No: 17816 Loss = 42
Batch No: 17817 Loss = 42
Batch No: 17818 Loss = 42
Batch No: 17

Batch No: 18100 Loss = 42
Batch No: 18101 Loss = 42
Batch No: 18102 Loss = 42
Batch No: 18103 Loss = 42
Batch No: 18104 Loss = 42
Batch No: 18105 Loss = 42
Batch No: 18106 Loss = 42
Batch No: 18107 Loss = 42
Batch No: 18108 Loss = 42
Batch No: 18109 Loss = 42
Batch No: 18110 Loss = 42
Batch No: 18111 Loss = 42
Batch No: 18112 Loss = 42
Batch No: 18113 Loss = 42
Batch No: 18114 Loss = 42
Batch No: 18115 Loss = 42
Batch No: 18116 Loss = 42
Batch No: 18117 Loss = 42
Batch No: 18118 Loss = 42
Batch No: 18119 Loss = 42
Batch No: 18120 Loss = 42
Batch No: 18121 Loss = 42
Batch No: 18122 Loss = 42
Batch No: 18123 Loss = 42
Batch No: 18124 Loss = 42
Batch No: 18125 Loss = 42
Batch No: 18126 Loss = 42
Batch No: 18127 Loss = 42
Batch No: 18128 Loss = 42
Batch No: 18129 Loss = 42
Batch No: 18130 Loss = 42
Batch No: 18131 Loss = 42
Batch No: 18132 Loss = 42
Batch No: 18133 Loss = 42
Batch No: 18134 Loss = 42
Batch No: 18135 Loss = 42
Batch No: 18136 Loss = 42
Batch No: 18137 Loss = 42
Batch No: 18

Batch No: 18418 Loss = 43
Batch No: 18419 Loss = 43
Batch No: 18420 Loss = 43
Batch No: 18421 Loss = 43
Batch No: 18422 Loss = 43
Batch No: 18423 Loss = 43
Batch No: 18424 Loss = 43
Batch No: 18425 Loss = 43
Batch No: 18426 Loss = 43
Batch No: 18427 Loss = 43
Batch No: 18428 Loss = 43
Batch No: 18429 Loss = 43
Batch No: 18430 Loss = 43
Batch No: 18431 Loss = 43
Batch No: 18432 Loss = 43
Batch No: 18433 Loss = 43
Batch No: 18434 Loss = 43
Batch No: 18435 Loss = 43
Batch No: 18436 Loss = 43
Batch No: 18437 Loss = 43
Batch No: 18438 Loss = 43
Batch No: 18439 Loss = 43
Batch No: 18440 Loss = 43
Batch No: 18441 Loss = 43
Batch No: 18442 Loss = 43
Batch No: 18443 Loss = 43
Batch No: 18444 Loss = 43
Batch No: 18445 Loss = 43
Batch No: 18446 Loss = 43
Batch No: 18447 Loss = 43
Batch No: 18448 Loss = 43
Batch No: 18449 Loss = 43
Batch No: 18450 Loss = 43
Batch No: 18451 Loss = 43
Batch No: 18452 Loss = 43
Batch No: 18453 Loss = 43
Batch No: 18454 Loss = 43
Batch No: 18455 Loss = 43
Batch No: 18

In [16]:
nlp.to_disk('models/')

In [17]:
test = []
for texts, annotation in test_data:
    doc = nlp.make_doc(texts)
    test.append(Example.from_dict(doc, annotation))   

In [18]:
score = nlp.evaluate(test)
score

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'cats_score': 0.9991020618999015,
 'cats_score_desc': 'macro AUC',
 'cats_micro_p': 0.9904540745239542,
 'cats_micro_r': 0.9901710775384835,
 'cats_micro_f': 0.9903125558135383,
 'cats_macro_p': 0.9909517775177864,
 'cats_macro_r': 0.9901203110960922,
 'cats_macro_f': 0.9902491712889673,
 'cats_macro_auc': 0.9991020618999015,
 'cats_f_per_type': {'label_1': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'label_2': {'p': 0.9987633743749664,
   'r': 0.9336080816203448,
   'f': 0.9650872817955113},
  'label_3': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'label_4': {'p': 0.9378990682495388,
   'r': 0.9972340960523007,
   'f': 0.9666569172272594},
  'label_5': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'label_6': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'label_7': {'p': 1.0, 'r': 1.0, 'f': 1.0}},
 'cats_auc_per_type': {'label_1': 1.0,
  'label_2': 0.9968454654160228,
  'label_3': 1.0,
  'label_4': 0.9968689678832875,
  'label_5': 1.0,
  'label_6': 1.0,
  'la

In [19]:
import json
print(json.dumps(score['cats_auc_per_type'], indent=4, sort_keys=True))

{
    "label_1": 1.0,
    "label_2": 0.9968454654160228,
    "label_3": 1.0,
    "label_4": 0.9968689678832875,
    "label_5": 1.0,
    "label_6": 1.0,
    "label_7": 1.0
}


In [20]:
texts = "KU016777X"
doc = nlp(texts)
doc.cats

{'label_1': 1.0223792742181104e-05,
 'label_2': 0.0010393396951258183,
 'label_3': 5.955985670880182e-06,
 'label_4': 0.000363984057912603,
 'label_5': 0.9999974966049194,
 'label_6': 2.4877604118955787e-06,
 'label_7': 3.3719413750077365e-06}

In [1]:
import pickle
pickle.dump(nlp, open( "nlp.pkl", "wb" ))

with open(r"nlp.pkl", "rb") as input_file:
    model = pickle.load(input_file)

NameError: name 'nlp' is not defined

In [22]:
texts = "123-23-3233"
doc = model(texts)
doc.cats

{'label_1': 0.9999899864196777,
 'label_2': 0.000612918462138623,
 'label_3': 1.7287959508394124e-06,
 'label_4': 0.00044287636410444975,
 'label_5': 1.2922603218612494e-06,
 'label_6': 5.318162834555551e-07,
 'label_7': 1.800245108540821e-08}

In [23]:
text = "123123123123"
doc = model(text)
doc.cats

{'label_1': 2.620249128781893e-09,
 'label_2': 0.98066645860672,
 'label_3': 1.240088522536098e-06,
 'label_4': 0.011949652805924416,
 'label_5': 4.790518204345062e-08,
 'label_6': 5.172234887140803e-05,
 'label_7': 1.503261682955781e-05}